# captura de información Ciencuadras.

Created on March 22 - 2022 </br>
@author: Alexander Robles


In [0]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

from sedona.register.geo_registrator import SedonaRegistrator

spark = (
    SparkSession.builder.master("local[*]")
    .appName("Sedona App")
    .config("spark.serializer", KryoSerializer.getName)
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName)
    .getOrCreate()
)

SedonaRegistrator.registerAll(spark)

In [0]:
%run ./config

In [0]:
import requests
import pandas as pd
import json

step_cap= {
    'ciencuadras': [raw_arriendo, raw_venta],
    #'ciencuadras': [raw_arriendo, raw_venta,raw_Proyectos_Nuevos],
}


def Ciencuadras_Request(url, raw):
    r = requests.post(url, data=raw)
    json_body = json.loads(raw)
    #name file
    name = json_body['pathurl']
    #_____
    data = json.loads(r.content)
    
    paginas = data['data']['totalPages']
    df = pd.DataFrame(json.loads(r.content)['data']['result'])
    for p in range(2,paginas):
        json_body['numberPaginator']=p
        raw_mod = json.dumps(json_body)
        r = requests.post(url, data=raw_mod)
        df_new = pd.DataFrame(json.loads(r.content)['data']['result'])
        df = pd.concat([df,df_new])
        #print(p)
    return df, name

In [0]:
from datetime import date
today = date.today()
fecha = str(today).replace("-", "")
for k,v in step_cap.items():
    if k == 'ciencuadras':
        for transaction in v:
            to_export, nombre = Ciencuadras_Request(url_ciencuadras, transaction)
            filename = '/dbfs/FileStore/tables/ciencuadras/{}_{}_ciencuadras.csv'.format(fecha, nombre)
            to_export.to_csv(filename, index=False)
            print(to_export.shape)

In [0]:
dbutils.fs.ls("/FileStore/tables/ciencuadras")